In [1]:
import csv
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time 
import pandas as pd
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException

In [2]:
from bs4 import BeautifulSoup

In [3]:
def get_url(product_type):
        
    '''generating the required url'''
    temp = 'https://www.amazon.in/s?k={}&ref=sr_pg_1'
    product_type = product_type.replace(' ','+')
    
    url = temp.format(product_type)
    
    url += '&page{}'
    
    return url

def records(item):
    atag = item.h2.a
    description = atag.text.strip()
    url = 'https://www.amazon.com'+ atag.get('href')
    try:
        price_parent = item.find('span', 'a-price')
        price = price_parent.find('span', 'a-offscreen').text
    except AttributeError:
        return
    try:
        rating = item.i.text
    except AttributeError:
        rating = ''
    result = (description, price, rating, url)
    return result 


def main(product_type):
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(executable_path='D:/projects/jupyter/chromedriver',options = options)
    
    record_list = []
    url = get_url(product_type)
    
    for page in range(1,21):
        driver.get(url.format(page))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.find_all('div', {'data-component-type': 's-search-result'})
        for item in results:
            record = records(item)
            if record:
                record_list.append(record)
                
    driver.close()
    
    with open('{}.csv'.format(product_type), 'w', newline='', encoding = 'utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Description', 'Price', 'Rating', 'Url'])
        writer.writerows(record_list)

In [4]:
main('smartphone')

In [5]:
pd.read_csv('smartphone.csv')

,Description,Price,Rating,Url
0,"Mi 11X Pro 5G (Lunar White, 8GB RAM, 128GB Sto...","₹39,999",3.9 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...
1,"Tecno Spark 7 (Magnet Black, 3GB RAM, 64 GB St...","₹8,499",3.9 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...
2,"Samsung Galaxy M31 (Ocean Blue, 8GB RAM, 128GB...","₹16,999",4.3 out of 5 stars,https://www.amazon.com/Samsung-Galaxy-Ocean-12...
3,"Samsung Galaxy M31 (Ocean Blue, 6GB RAM, 128GB...","₹14,999",4.3 out of 5 stars,https://www.amazon.com/Samsung-Galaxy-Ocean-12...
4,"Redmi 9A (Nature Green, 2GB RAM, 32GB Storage)...","₹6,999",4.2 out of 5 stars,https://www.amazon.com/Redmi-9A-2GB-32GB-Stora...
...,...,...,...,...
435,"Redmi 9 (Carbon Black, 4GB RAM, 64GB Storage) ...","₹8,999",4.2 out of 5 stars,https://www.amazon.com/Redmi-Carbon-Black-64GB...
436,Redmi 9A (Midnight Black 2GB RAM 32GB Storage)...,"₹6,999",4.2 out of 5 stars,https://www.amazon.com/Redmi-9A-Midnight-2GB-3...
437,"Redmi 9 (Sporty Orange, 4GB RAM, 64GB Storage)...","₹8,999",4.2 out of 5 stars,https://www.amazon.com/Redmi-Sporty-Orange-64G...
438,"iQOO Z3 5G (Cyber Blue, 8GB RAM, 128GB Storage...","₹20,990",4.3 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...
